In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D , MaxPool2D ,AveragePooling2D, Dense , Dropout , Flatten , Input , GlobalAveragePooling2D, Concatenate

In [12]:
def StemBlock(prev_layer):
    x = Conv2D(filters = 64, kernel_size = (7,7), strides=2 , padding='valid' , activation='relu' )(prev_layer)
    x = MaxPool2D(pool_size=(3,3) , strides=2 )(x)
    x = Conv2D(filters = 64, kernel_size = (1,1), strides=1 , padding='same' , activation='relu' )(x)
    x = Conv2D(filters = 192, kernel_size = (3,3), strides=1 , padding='same' , activation='relu' )(x)
    x = MaxPool2D(pool_size=(3,3) , strides=2 )(x)
    return x

In [13]:
def InceptionBlock(previous_layer , nbr_f1 , nbr_f2_1 , nbr_f2_2 , nbr_f3_1 , nbr_f3_2 , nbr_f4) :

    #Path 1
    path1 = Conv2D(filters=nbr_f1, kernel_size = (1,1), padding='same' , activation='relu')(previous_layer)

    #Path 2
    path2 = Conv2D(filters=nbr_f2_1, kernel_size = (1,1), padding='same' , activation='relu')(previous_layer)
    path2 = Conv2D(filters=nbr_f2_2, kernel_size = (3,3), padding='same' , activation='relu')(path2)

    #Path 3
    path3 = Conv2D(filters=nbr_f3_1, kernel_size = (1,1), padding='same' , activation='relu')(previous_layer)
    path3 = Conv2D(filters=nbr_f3_1, kernel_size = (5,5), padding='same' , activation='relu')(path3)

    #Path 4
    path4 = MaxPool2D(pool_size=(3,3) , strides=(1,1) , padding='same') (previous_layer)
    path4 = Conv2D(filters=nbr_f4, kernel_size = (1,1), padding='same' , activation='relu')(path4)

    output_Layer = Concatenate(axis=-1)([path1 , path2 , path3 , path4])
    return output_Layer

In [14]:
def Auxiliary_classifier(prev_layer):
    x = AveragePooling2D(pool_size = (5,5), strides = 3)(prev_layer)
    x = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(x)
    x = Flatten()(x)
    x = Dense(1024, activation = 'relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(1000, activation = 'softmax')(x)
    return x

In [19]:
def InceptionV1():
    input_layer = Input(shape = (224, 224, 3))

    x1 = StemBlock(input_layer)

    x1 = InceptionBlock(previous_layer=x1, nbr_f1=64, nbr_f2_1=96, nbr_f2_2=128, nbr_f3_1=16, nbr_f3_2=32, nbr_f4=32)
    x1 = InceptionBlock(previous_layer=x1, nbr_f1=128, nbr_f2_1=128, nbr_f2_2=192, nbr_f3_1=32, nbr_f3_2=96, nbr_f4=64)
    x1 = MaxPool2D(pool_size=(3,3) , strides=2 )(x1)
    x1 = InceptionBlock(previous_layer=x1, nbr_f1=192, nbr_f2_1=96, nbr_f2_2=208, nbr_f3_1=16, nbr_f3_2=48, nbr_f4=64)

    # Auxiliary classifier 1
    x2 = Auxiliary_classifier(x1)
    # End Auxiliary classifier 1

    x1 = InceptionBlock(previous_layer=x1, nbr_f1=160, nbr_f2_1=112, nbr_f2_2=224, nbr_f3_1=24, nbr_f3_2=64, nbr_f4=64)
    x1 = InceptionBlock(previous_layer=x1, nbr_f1=128, nbr_f2_1=128, nbr_f2_2=256, nbr_f3_1=24, nbr_f3_2=64, nbr_f4=64)
    x1 = InceptionBlock(previous_layer=x1, nbr_f1=112, nbr_f2_1=144, nbr_f2_2=288, nbr_f3_1=32, nbr_f3_2=64, nbr_f4=64)

    # Auxiliary classifier 2
    x3 = Auxiliary_classifier(x1)
    # End Auxiliary classifier 1

    x1 = InceptionBlock(previous_layer=x1, nbr_f1=256, nbr_f2_1=160, nbr_f2_2=320, nbr_f3_1=32, nbr_f3_2=128, nbr_f4=128)
    x1 = MaxPool2D(pool_size=(3,3) , strides=2)(x1)
    x1 = InceptionBlock(previous_layer=x1, nbr_f1=256, nbr_f2_1=160, nbr_f2_2=320, nbr_f3_1=32, nbr_f3_2=128, nbr_f4=128)
    x1 = InceptionBlock(previous_layer=x1, nbr_f1=384, nbr_f2_1=192, nbr_f2_2=384, nbr_f3_1=48, nbr_f3_2=128, nbr_f4=128)

    x1 = GlobalAveragePooling2D(name = 'GAPL')(x1)
    x1 = Dropout(0.4)(x1)
    x1 = Dense(units=1000, activation='relu')(x1)
    x1 = Dense(units=1000, activation='softmax')(x1)

    # model = Model(input_layer, [x1 , x2 , x3] , name='InceptionV1')
    model = Model(input_layer, x1 , name='InceptionV1') # not using auxillary classifier

    return model


In [20]:
model = InceptionV1()
model.summary()

Model: "InceptionV1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_137 (Conv2D)         (None, 109, 109, 64)         9472      ['input_5[0][0]']             
                                                                                                  
 max_pooling2d_32 (MaxPooli  (None, 54, 54, 64)           0         ['conv2d_137[0][0]']          
 ng2D)                                                                                            
                                                                                                  
 conv2d_138 (Conv2D)         (None, 54, 54, 64)           4160      ['max_pooling2d_32[0